In [3]:
import numpy as np
import sys
from ipycanvas import Canvas
from tqdm import tqdm
sys.path.append('../src')
from util import draw, grab_plot, plot, RunTime, VideoWriter

In [4]:
from ipycanvas import hold_canvas

In [ ]:
def update():
    global farm, canvas, x, v, a

    x, v = farm(x, v, a)
    draw(x, canvas, CANVAS_SIZE, ant_size=5)

In [ ]:
class Studio:
    def __init__(self, model, interval=.02, **kwargs):
        self.model = model(**kwargs)
        self.interval = interval

        self.timer = None
        self.running = False

    def step(self):
        self.running = False
        self.start()
        self.model()
        
    def update(self):
        pass

    def start(self):
        display(self.canvas)

        if self.runtime is not None:
            self.runtime.stop()

        self.runtime = RunTime(self.update, self.interval)

In [ ]:
def draw_rect(farm, xy, width, height, color):
    fill_style = farm.fill_style
    farm.fill_style = color
    farm.fill_rect(*xy, width, height)
    farm.fill_style = fill_style

def draw_rects(farm, xy, width, height, color):
    fill_style = farm.fill_style
    farm.fill_style = color
    farm.fill_rects(*xy.T, width, height)
    farm.fill_style = fill_style

def draw(farm, ants, walls=None, farm_size=500, farm_color='green', ant_size=5., ant_color='red', wall_color='white'):
    with hold_canvas():
        farm.clear()
        farm.fill_style = 'red'
        draw_rect(farm, (0, 0), farm_size, farm_size, farm_color)
        # farm.fill_style = ant_color
        farm.fill_circles(*ants.T, ant_size)
        
        # if walls is not None:
        #     # farm.fill_style = wall_color
        #     farm.fill_styled_rects(*walls.T, color=wall_color)

In [8]:
FARM_SIZE = 500
N_ANTS = 10

farm = Canvas(width=FARM_SIZE, height=FARM_SIZE)
# ants = .5*FARM_SIZE*np.ones((N_ANTS, 2))
ants = FARM_SIZE*np.random.random((N_ANTS, 2))
walls = np.array([[0, 10, 50, 10]])

draw(farm, ants, walls)
display(farm)

Canvas(width=500)

In [2]:
CANVAS_SIZE = 500

def update():
    global farm, canvas, x, v, a

    x, v = farm(x, v, a)
    draw(x, canvas, CANVAS_SIZE, ant_size=5)

def toggle(*_):
    global runtime 

    if runtime.running:
        runtime.stop()
    else:
        runtime.start()

def simulate(function, interval=.02):
    global runtime, canvas, x

    display(canvas)
    draw(x, canvas, CANVAS_SIZE, ant_size=5)

    try:
        runtime.stop()
    except:
        pass

    runtime = RunTime(function, interval)

def record(n_steps=500):
    global farm, x, v

    with VideoWriter(size=CANVAS_SIZE) as vid:
        for _ in tqdm(range(n_steps), desc='Ant Farm'):
            plot(x)
            vid.write(grab_plot())
            x, v = farm(x, v)

In [3]:


class KinematicsFarm:
    def __init__(self, dt=.2, dv=.85, figsize=500):
        self.dt = dt
        self.dv = dv
        self.figsize = figsize

    def __call__(self, x, v, a):
        in_wall = False

        for i in range(x.shape[0]):
            if x[i, 0] > self.figsize - 5.:
                v[i, 0] = -self.dv*np.abs(v[i, 0])
                v[i, 1] = self.dv*v[i, 1]
                in_wall = True
            elif x[i, 0] < 5.:
                v[i, 0] = self.dv*np.abs(v[i, 0])
                v[i, 1] = self.dv*v[i, 1]
                in_wall = True
            else:
                v[i, 0] = v[i, 0] + a[i, 0]*self.dt
                
            if x[i, 1] > self.figsize - 5.:
                v[i, 0] = self.dv*v[i, 0]
                v[i, 1] = -self.dv*np.abs(v[i, 1])
                in_wall = True
            elif x[i, 1] < 5.:
                v[i, 0] = self.dv*v[i, 0]
                v[i, 1] = self.dv*np.abs(v[i, 1])
                in_wall = True
            else:
                v[i, 1] = v[i, 1] + a[i, 1]*self.dt

        x = x + v*self.dt + (not in_wall)*.5*self.dt**2

        return x, v

In [5]:
walls = np.array([[[0, 0], [0, CANVAS_SIZE]],
                  [[0, 0], [CANVAS_SIZE, 0]],
                  [[0, CANVAS_SIZE], [CANVAS_SIZE, CANVAS_SIZE]],
                  [[CANVAS_SIZE, 0], [CANVAS_SIZE, CANVAS_SIZE]]])

walls

# check if line from current position to next intersects with wall

array([[[  0,   0],
        [  0, 500]],

       [[  0,   0],
        [500,   0]],

       [[  0, 500],
        [500, 500]],

       [[500,   0],
        [500, 500]]])

In [5]:
x1 = x
x2 = x + v*.1

In [6]:
x1

array([[224.869024  , 369.69056063],
       [ 37.87564677, 443.3471494 ],
       [157.83075586, 235.04619621],
       [301.9314621 , 178.07428411],
       [ 89.44501709, 495.95509503],
       [143.42016353, 461.82559717],
       [112.74599102, 135.96848178],
       [160.28096807, 140.27126248],
       [238.56430762, 146.25168234],
       [192.18344342, 158.77601788]])

In [7]:
x2

array([[224.28786546, 374.45880064],
       [ 33.04463874, 449.7894028 ],
       [155.72600031, 236.75433703],
       [303.1017226 , 178.48760875],
       [ 85.78604021, 503.59298356],
       [140.98789452, 468.68781529],
       [109.61658173, 135.42485637],
       [158.23189916, 139.82542754],
       [238.29440552, 145.94176603],
       [190.85943077, 158.75074556]])

In [4]:
N_ANTS = 10

farm = KinematicsFarm(figsize=CANVAS_SIZE)
x = .5*CANVAS_SIZE*np.ones((N_ANTS, 2))
# v = np.zeros_like(x)
v = -100.*np.random.random(x.shape) + 50.
a = np.array([[0., 9.8],]*N_ANTS)
canvas = Canvas(width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas.on_mouse_down(toggle)

simulate(update)

Canvas(width=500)